In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Data/Beer

/content/drive/MyDrive/Data/Beer


In [3]:
# List relevant data directory paths
import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./beer_reviews.csv
./beer_data_set.csv
./Brewery Name Fuzzy Match List.csv
./beer_profile_and_ratings.csv
./Beer Name Fuzzy Match List.csv
./Beer Descriptors Simplified.xlsx


# Introduction

This notebook outlines how the [Beer Profile and Ratings Dataset](https://www.kaggle.com/ruthgn/beer-profile-and-ratings-data-set) was created--specifically the data wrangling process in integrating the existing datasets below:
* [1.5 Million Beer Reviews](https://www.kaggle.com/rdoume/beerreviews)
* [Beer Tasting Profiles Dataset](https://www.kaggle.com/stephenpolozoff/top-beer-information)

The purpose of the data integration is to create a new dataset that contains consumer ratings and reviews for different brews, combined with their detailed tasting profiles.

## Imports and Configuration

In [5]:
!pip3 install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
# Set Data Directory
data_dir = Path('/content/drive/MyDrive/Data/Beer')

## Beer Reviews Data

Source: [1.5 Million Beer Reviews](https://www.kaggle.com/rdoume/beerreviews) by Tanya Cashorali (uploaded by [Datadoume](https://www.kaggle.com/rdoume)).

Our first dataset contains 1.5 million beer reviews from *BeerAdvocate*. Each individual review includes appearance, aroma, palate, and taste scores, as well as an overall review score.

In [10]:
# Load first data set - Beer Reviews
df_reviews = pd.read_csv(data_dir / 'beerreviews/beer_reviews.csv')
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [11]:
df_reviews.head(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


We need to aggregate the data because it contains multiple reviews for a single brew. We'll do this getting the average scores of each brew.

In [12]:
# Get average scores for each brew and call new dataframe `df_scores`
df_scores = df_reviews.drop(['brewery_id', 
                             'review_time', 
                             'review_profilename', 
                             'beer_style', 
                             'beer_beerid', 
                             'beer_abv'], 
                            axis=1).groupby(by=['brewery_name', 
                                                'beer_name'], 
                                            as_index = False).mean()

# Get count of reviews per brew
df_scores_count = df_reviews.drop(['brewery_id', 'review_time', 'review_profilename', 'beer_style', 'beer_beerid', 'beer_abv'], axis=1).groupby(by=['brewery_name', 'beer_name'], as_index=False).count()
df_scores_count = df_scores_count['review_overall']

# Combine information into one dataframe 
df_scores['number_of_reviews'] = df_scores_count
df_scores.head()

,brewery_name,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews
0,'t Hofbrouwerijke,Blondelle,4.000000,4.500000,3.500000,4.000000,4.000000,1
1,'t Hofbrouwerijke,Bosprotter,3.722222,3.833333,3.666667,3.611111,3.777778,9
2,'t Hofbrouwerijke,Hof Korvatunturi,3.750000,4.000000,3.750000,3.500000,3.750000,2
3,'t Hofbrouwerijke,Hofblues,3.812500,3.812500,4.000000,3.625000,3.593750,16
4,'t Hofbrouwerijke,Hofdraak,3.357143,3.428571,3.714286,3.142857,3.428571,7


## Beer Profile Data

Source: [Beer Tasting Profiles Dataset](https://www.kaggle.com/stephenpolozoff/top-beer-information) by [sp1222](https://www.kaggle.com/stephenpolozoff).

The second dataset we're going to use contains tasting profiles of up to 50 top-rated beers across 112 styles, 5558 beers in total, scrapped from *BeerAdvocate*. The first ten columns provide information on the beer provided by the source, along with contributed information like a unique key for each beer and style. The last eleven columns represent the tasting profile features of the beer, and are defined by word counts found in up to 25 reviews of each beer. The assumption is that people writing reviews are more than likely describing what they do experience rather than what they do not. Note that one or two styles did not have 50 beers to collect information on, and that few beers may have had limited or no reviews available to define features with.

In [13]:
# Load second data set - Beer Profile
df_profile = pd.read_csv(data_dir / 'top-beer-information/beer_data_set.csv')
df_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5558 entries, 0 to 5557
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         5556 non-null   object 
 1   key          5558 non-null   int64  
 2   Style        5558 non-null   object 
 3   Style Key    5558 non-null   int64  
 4   Brewery      5558 non-null   object 
 5   Description  5558 non-null   object 
 6   ABV          5558 non-null   float64
 7   Ave Rating   5558 non-null   float64
 8   Min IBU      5558 non-null   int64  
 9   Max IBU      5558 non-null   int64  
 10  Astringency  5558 non-null   int64  
 11  Body         5558 non-null   int64  
 12  Alcohol      5558 non-null   int64  
 13  Bitter       5558 non-null   int64  
 14  Sweet        5558 non-null   int64  
 15  Sour         5558 non-null   int64  
 16  Salty        5558 non-null   int64  
 17  Fruits       5558 non-null   int64  
 18  Hoppy        5558 non-null   int64  
 19  Spices

In [14]:
df_profile.head()

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218


# Data Cleaning

Our goal is to match as many brews from `df_profile` with the available beer ratings data from `df_scores`.

To create a unique identifier for every single brew on both data frames, we need to concatenate the brewery name and the beer name. The `Amber` beer by `Alaskan Brewing Co.` on the first row of `df_profile` above, for example, will be identified as `Alaskan Brewing Co. Amber`.

Before we proceed with that, however, there are several checks we need to perform to make sure that the concatenation will yield unique identifiers that are (somewhat) consistent accross both data frames for optimum results:
1. Check rows with missing data on `Name` column on `df_profile`.
2. Match brewery names from both data frames.

In [15]:
# Display `df_profile` rows with null values (missing beer names)
df_profile[df_profile['Name'].isnull()]

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
1803,NaN,3504,Kvass,81,Monastyrskiy Kvas,Notes:,1.5,3.07,0,0,...,34,4,15,84,16,1,33,14,7,68
2150,NaN,2401,Lager - European Pale,57,Stella Artois,Notes:,5.0,3.11,18,25,...,14,10,20,19,3,0,1,26,3,26


In [16]:
# Look up data on 'Monastyrskiy Kvas' from `df_scores`
df_scores[df_scores['brewery_name']=='Monastyrskiy Kvas']


# Highlight important row
df_scores[df_scores['brewery_name']=='Monastyrskiy Kvas'].style.apply(
    lambda x: ['background: lightgreen' if x.name == 40781 else '' for i in x], 
    axis=1)

,brewery_name,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews
40779,Monastyrskiy Kvas,Monastyrskiy Black Currant Kvas,3.500000,4.100000,3.600000,3.100000,3.600000,5
40780,Monastyrskiy Kvas,Monastyrskiy Cranberry Kvas,3.500000,3.833333,3.500000,3.000000,3.500000,3
40781,Monastyrskiy Kvas,Monastyrskiy Kvas,3.062500,3.187500,2.937500,2.812500,3.062500,8
40782,Monastyrskiy Kvas,Monastyrskiy Okroshka Kvas,3.625000,3.500000,3.250000,2.875000,3.625000,4
40783,Monastyrskiy Kvas,Verkin Kvas,3.125000,3.500000,3.125000,3.000000,2.750000,4
40784,Monastyrskiy Kvas,Verkin Pohmilny Kvas,3.500000,3.500000,2.500000,3.000000,3.500000,1


In [17]:
# Look up data on 'Stella Artois' from `df_scores`
df_scores[df_scores['brewery_name']=='Stella Artois']


# Highlight important row
df_scores[df_scores['brewery_name']=='Stella Artois'].style.apply(
    lambda x: ['background: lightgreen' if x.name == 55651 else '' for i in x], 
    axis=1)

,brewery_name,beer_name,review_overall,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews
55635,Stella Artois,Artois Bock,3.277778,3.222222,3.555556,3.111111,3.111111,9
55636,Stella Artois,Atlas,1.857143,2.357143,2.714286,2.000000,1.928571,7
55637,Stella Artois,Bergenbrau,3.500000,3.000000,2.750000,3.250000,4.000000,2
55638,Stella Artois,Club De Stella Artois,3.384615,2.826923,3.240385,3.221154,3.134615,52
55639,Stella Artois,Dutch Gold Imported,2.833333,2.000000,2.833333,2.500000,2.166667,3
55640,Stella Artois,Eiken Artois,2.875000,2.625000,3.125000,2.500000,2.500000,4
55641,Stella Artois,Ginder Ale,3.666667,3.500000,3.666667,3.500000,3.333333,3
55642,Stella Artois,Horse-Ale,4.000000,4.166667,4.000000,4.333333,3.833333,3
55643,Stella Artois,La Becasse Framboise,2.666667,2.916667,3.416667,3.000000,2.666667,6
55644,Stella Artois,La Becasse Gueuze,3.166667,3.000000,3.000000,3.166667,2.500000,3


From looking at the tables, we know that the appropriate review data match for each brew lies in the row with highest the `number_of_reviews` on `df_scores` (see highlighted rows above). Let's go ahead and apply an identical label format for the two brews on `df_profile`:

In [18]:
# Edit `Name` column for `Stella Artois` and `Monastyrskiy Kvas` on `df_profile`
df_profile.loc[[1803, 2150],'Name'] = df_profile.loc[[1803, 2150],'Brewery']

## Matching brewery names

Now we're going to match brewery names from both data frames.

In [19]:
# Create new column on `df_profile` indicating whether the brewery name exists on `df_scores`
df_profile['brewery_review_exists'] = np.where(df_profile['Brewery'].isin(list(df_scores['brewery_name'].unique())), 1, 0)


# Formatting for better display
slice_ = 'brewery_review_exists'

def highlight_indicator(val):
    pink = 'background-color: pink' if val < 1 else ''
    return pink

df_profile.head(10).style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)\
                         .applymap(highlight_indicator, subset=[slice_])

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, with just enough hop backing to make this beautiful amber colored ""alt"" style beer notably well balanced.",5.300000,3.650000,25,50,13,32,9,47,74,33,0,33,57,8,111,1
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is also known as “Stickebier” – German slang for “secret brew”. Long Trail Double Bag was originally offered only in our brewery taproom as a special treat to our visitors. With an alcohol content of 7.2%, please indulge in moderation. The Long Trail Brewing Company is proud to have Double Bag named Malt Advocate’s “Beer of the Year” in 2001. Malt Advocate is a national magazine devoted to “expanding the boundaries of fine drinks”. Their panel of judges likes to keep things simple, and therefore of thousands of eligible competitors they award only two categories: “Imported” and “Domestic”. It is a great honor to receive this recognition.33 IBU",7.200000,3.900000,25,50,12,57,18,33,55,16,0,24,35,12,84,1
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,"Notes:Long Trail Ale is a full-bodied amber ale modeled after the “Alt-biers” of Düsseldorf, Germany. Our top fermenting yeast and cold finishing temperature result in a complex, yet clean, full flavor. Originally introduced in November of 1989, Long Trail Ale beer quickly became, and remains, the largest selling craft-brew in Vermont. It is a multiple medal winner at the Great American Beer Festival.25 IBU",5.000000,3.580000,25,50,14,37,6,42,43,11,0,10,54,4,62,1
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.500000,4.150000,25,50,13,55,31,47,101,18,1,49,40,16,119,0
4,Scurry,255,Altbier,8,Off Color Brewing,"Notes:Just cause it's dark and German doesn't mean it's an alt. Based on the obscure Kottbusser style of beer, this surprisingly dry beer retains all the aroma and nuance of honey (from bees) and molasses (not from bees) creating a perception of sweetness through cool, controlled fermentation. Because we're sweet enough damn it. 18 IBU",5.300000,3.670000,25,50,21,69,10,63,120,14,0,19,36,15,218,0
5,Sleigh'r Dark Doüble Alt Ale,256,Altbier,8,Ninkasi Brewing Company,"Notes:Called 'Dark Double Alt' on the label.Seize the season with Sleigh'r. Layers of deeply toasted malt are balanced by just enough hop bitterness to make it deceivingly drinkable. Paired with a dry finish, Sleigh’r is anything but your typical winter brew.An Alt ferments with Ale yeast at colder lagering temperatures. This effect gives Alts a more refined, crisp lager-like flavor than traditional ales. The Alt has been “Ninkasified” raising the ABV and IBUs. Sleigh'r has a deep, toasted malt flavor that finishes dry and balanced.50 IBU",7.200000,3.780000,25,50,25,51,26,44,45,9,1,11,51,20,95,1
6,Sticke,257,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,6.000000,4.100000,25,50,22,45,13,46,62,25,1,34,60,4,103,0
7,Okto Festival Ale,258,Altbier,8,Widmer Brothers Brewing Company,Notes:Classified as a German Altbier by the Widmer Brothers; brewed with an (ale) Alt yeast strain.availability: early Aug. - late Oct.bitterness : 28 IBUoriginal gravity: 13.5° PLATO,5.300000,3.460000,25,50,28,40,3,40,58,29,0,36,54,8,97,1
8,Southampton Altbier,259,Altbier,8,Southampton Publick House,Notes:,5.000000,3.600000,25,50,18,49,5,37,73,22,0,21,37,4,98,1
9,Copper,260,Altbier,8,The Olde Mecklenburg Brewery,"Notes:OMB’s flagship, Copper, is an authentic Düsseldorf style “Altbier” or Amber Ale. There are no crazy flavors or gimmicks with Copper – its full-flavored start transitions to a smooth, balanced, easy-drinking finish. It’s the beer that tastes like beer.",4.800000,4.100000,25,50,25,35,4,38,39,13,1,8,60,16,97,1


In [20]:
# Create new data frame (`brewery_no_scores`) listing breweries on `df_profile` with no exact match on `df_scores`
brewery_no_scores = pd.DataFrame(df_profile[df_profile['brewery_review_exists']==0]['Brewery'].unique()).set_axis(['Brewery'], axis=1)
brewery_no_scores

,Brewery
0,Uerige Obergärige Hausbrauerei
1,Off Color Brewing
2,Victory Brewing Company - Downingtown
3,Hops & Grain Brewing
4,Union Craft Brewing Company
...,...
538,Dust Bowl Brewing Company - Downtown Taproom
539,Griffin Claw Brewing Company
540,Anthem Brewing Company
541,Aslin Beer Company


We want to check if some of the breweries on the previous list *actually* have corresponding data on `df_scores` without having the *exact* same brewery name labels (caused by typos, character errors, inconsistent labelling, etc.). This is where the Python library [FuzzyWuzzy](https://github.com/seatgeek/fuzzywuzzy) comes in handy. We will create a new function to find "fuzzy" matches for the remaining 543 breweries on `df_profile`.

In [21]:
# Fuzzy matching
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [22]:
# Create new column on `df_scores` indicating whether the brewery name exists on `df_profile`
df_scores['brewery_profile_exists'] = np.where(df_scores['brewery_name'].isin(list(df_profile['Brewery'].unique())), 1, 0)

# Create new data frame (`brewery_no_profile`) listing breweries on `df_scores` with no exact match on `df_profile`
brewery_no_profile = pd.DataFrame(df_scores[df_scores['brewery_profile_exists']==0]['brewery_name'].unique()).set_axis(['Brewery'], axis=1)

In [23]:
# # (Uncomment to let pandas display all rows and column content for all data frames)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

# # (Default setting)
# pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_colwidth', 50)

In [24]:
# # Get fuzzy matches for 543 breweries
# # Warning: Takes a long time to run
# fuzzy_match_brewery = fuzzy_merge(brewery_no_scores, brewery_no_profile, 'Brewery', 'brewery_name', threshold=88, limit=1)
# fuzzy_match_brewery.to_csv(your_data_dir / 'Brewery Fuzzy Match List.csv', index=False)

After manually going through the fuzzy matches, we found 87 correct matches and saved them as a .csv file called "Brewery Name Fuzzy Match List.csv" (to save time from having to run the code each time and re-do some manual corrections). We're going to replace the brewery names in `df_scores` with their equivalents from `df_profile`.

In [33]:
# Load table containing saved brewery name matches ("Brewery Name Fuzzy Match List.csv")
fuzzy_match_brewery = pd.read_csv(data_dir / "beer-profile-and-ratings-data-set/Brewery Name Fuzzy Match List.csv")
fuzzy_match_brewery

,Brewery,matches
0,Žatecký Pivovar,Žatecký Pivovar
1,Abbey Brewing Co. (Monastery Of Christ In The ...,Abbey Brewing Co.
2,Aktienbrauerei Kaufbeuren AG,Aktien-Brauerei Kaufbeuren AG
3,Alltech's Lexington Brewing and Distilling Co.,Alltech’s Lexington Brewing and Distilling Co.
4,Asia Pacific Breweries Limited,Asia Pacific Breweries Ltd.
...,...,...
82,Urban Chestnut Brewing Company - Midtown Brewe...,Urban Chestnut Brewing Company
83,Victory Brewing Company - Downingtown,Victory Brewing Company
84,Volfas Engelman,Vofas-Engelman
85,Water Street Brewing Co.,Water Street Brewing & Ale House


In [34]:
# Save table as dictionary
fuzzy_match_dict = dict(zip(fuzzy_match_brewery['Brewery'], fuzzy_match_brewery['matches']))

# Replace relevant brewery names in `df_profile`
df_profile['Brewery'] = df_profile['Brewery'].replace(fuzzy_match_dict)

In [35]:
# Additionally, manually replace "Yuengling Brewery " with "Yuengling Brewery" (no space) in both `df_profile` and `df_scores`
df_profile['Brewery'] = df_profile['Brewery'].replace({'Yuengling Brewery ': 'Yuengling Brewery'})
df_scores['brewery_name'] = df_scores['brewery_name'].replace({'Yuengling Brewery ': 'Yuengling Brewery'})

In [36]:
# Correcting 'brewery_review_exists' indicator column values on `df_profile` after using fuzzy matches
df_profile['brewery_review_exists'] = np.where(df_profile['Brewery'].isin(list(df_scores['brewery_name'].unique())), 1, 0)

# Correcting 'brewery_profile_exists' indicator column values on `df_scores` after using fuzzy matches
df_scores['brewery_profile_exists'] = np.where(df_scores['brewery_name'].isin(list(df_profile['Brewery'].unique())), 1, 0)

To speed up all upcoming data processing, we will remove rows on `df_scores` with brewery names that don't exist in `df_profiles` at all. Similarly, we're also removing rows on `df_profiles` with breweries that don't exist `df_scores`. We are removing these brews because we won't have their appropriate data points for our desired integration.

In [37]:
# Remove all brews from breweries that are not listed in either `df_profile` or `df_scores`
df_scores.drop(df_scores[df_scores['brewery_profile_exists']==0].index, axis=0, inplace=True)
df_profile.drop(df_profile[df_profile['brewery_review_exists']==0].index, axis=0, inplace=True)

## Matching beer names (brews)

As mentioned previously, in order to integrate information from both datasets, we need to create a unique identifier for every single brew. We will concatenate the brewery name and beer name to create this unique identifier.

In [38]:
# Create new identifier column in `df_scores` called 'beer_name_full'
# Fill column by concatenating `brewery_name` and `beer_name`
df_scores['beer_name_full'] = df_scores['brewery_name'] + ' ' + df_scores['beer_name']

# Check if all generated brew names in df_scores are unique:
df_scores['beer_name_full'].nunique() == len(df_scores.index)

True

In [39]:
# Create new identifier column in `df_profile` called 'Beer Name (Full)' 
# Fill column by concatenating `Brewery` and `Name`
df_profile['Beer Name (Full)'] = df_profile['Brewery'] + ' ' + df_profile['Name']

# Check if all generated brew names in `df_profile` are unique:
df_profile['Beer Name (Full)'].nunique() == len(df_profile.index)

False

In [40]:
# Check duplicated brew name
df_profile[df_profile['Beer Name (Full)'].duplicated()]

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,...,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists,Beer Name (Full)
4056,Crazy Jackass Ale,594,Rye Beer - Roggenbier,15,Sweetwater Tavern & Brewery,Notes:,5.5,3.84,10,20,...,0,0,0,0,0,0,0,0,1,Sweetwater Tavern & Brewery Crazy Jackass Ale


In [41]:
# List rows with duplicated brew name
df_profile[df_profile['Beer Name (Full)']=='Sweetwater Tavern & Brewery Crazy Jackass Ale']


# Highlight row with error
df_profile[df_profile['Beer Name (Full)']=='Sweetwater Tavern & Brewery Crazy Jackass Ale'].style.apply(
    lambda x: ['background: pink' if x.name == 4056 else '' for i in x], 
    axis=1)

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists,Beer Name (Full)
4030,Crazy Jackass Ale,568,Rye Beer - Roggenbier,15,Sweetwater Tavern & Brewery,Notes:,6.000000,3.960000,10,20,2,5,1,6,4,3,0,5,8,4,26,1,Sweetwater Tavern & Brewery Crazy Jackass Ale
4056,Crazy Jackass Ale,594,Rye Beer - Roggenbier,15,Sweetwater Tavern & Brewery,Notes:,5.500000,3.840000,10,20,0,0,0,0,0,0,0,0,0,0,0,1,Sweetwater Tavern & Brewery Crazy Jackass Ale


In [42]:
# Remove duplicate row containing error (missing data)
df_profile.drop(4056, inplace=True)

It's time to match our brew names (unique identifiers) from `df_profile` with the ones that exist on `df_scores`. We're going to follow a process similar to the one we've applied on the brewery names earlier.

In [43]:
# Create new column on `df_profile` indicating whether the complete brew name exists on `df_scores`
df_profile['beer_review_exists'] = np.where(df_profile['Beer Name (Full)'].isin(list(df_scores['beer_name_full'])), 1, 0)


# Formatting for better display
slice_ = 'beer_review_exists'

df_profile.head(10).style.set_properties(**{'background-color': '#ffffb3'}, subset=slice_)\
                         .applymap(highlight_indicator, subset=[slice_])

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,brewery_review_exists,Beer Name (Full),beer_review_exists
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, with just enough hop backing to make this beautiful amber colored ""alt"" style beer notably well balanced.",5.300000,3.650000,25,50,13,32,9,47,74,33,0,33,57,8,111,1,Alaskan Brewing Co. Amber,0
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is also known as “Stickebier” – German slang for “secret brew”. Long Trail Double Bag was originally offered only in our brewery taproom as a special treat to our visitors. With an alcohol content of 7.2%, please indulge in moderation. The Long Trail Brewing Company is proud to have Double Bag named Malt Advocate’s “Beer of the Year” in 2001. Malt Advocate is a national magazine devoted to “expanding the boundaries of fine drinks”. Their panel of judges likes to keep things simple, and therefore of thousands of eligible competitors they award only two categories: “Imported” and “Domestic”. It is a great honor to receive this recognition.33 IBU",7.200000,3.900000,25,50,12,57,18,33,55,16,0,24,35,12,84,1,Long Trail Brewing Co. Double Bag,1
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,"Notes:Long Trail Ale is a full-bodied amber ale modeled after the “Alt-biers” of Düsseldorf, Germany. Our top fermenting yeast and cold finishing temperature result in a complex, yet clean, full flavor. Originally introduced in November of 1989, Long Trail Ale beer quickly became, and remains, the largest selling craft-brew in Vermont. It is a multiple medal winner at the Great American Beer Festival.25 IBU",5.000000,3.580000,25,50,14,37,6,42,43,11,0,10,54,4,62,1,Long Trail Brewing Co. Long Trail Ale,1
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Notes:,8.500000,4.150000,25,50,13,55,31,47,101,18,1,49,40,16,119,1,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige Doppelsticke,0
5,Sleigh'r Dark Doüble Alt Ale,256,Altbier,8,Ninkasi Brewing Company,"Notes:Called 'Dark Double Alt' on the label.Seize the season with Sleigh'r. Layers of deeply toasted malt are balanced by just enough hop bitterness to make it deceivingly drinkable. Paired with a dry finish, Sleigh’r is anything but your typical winter brew.An Alt ferments with Ale yeast at colder lagering temperatures. This effect gives Alts a more refined, crisp lager-like flavor than traditional ales. The Alt has been “Ninkasified” raising the ABV and IBUs. Sleigh'r has a deep, toasted malt flavor that finishes dry and balanced.50 IBU",7.200000,3.780000,25,50,25,51,26,44,45,9,1,11,51,20,95,1,Ninkasi Brewing Company Sleigh'r Dark Doüble Alt Ale,1
6,Sticke,257,Altbier,8,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Notes:,6.000000,4.100000,25,50,22,45,13,46,62,25,1,34,60,4,103,1,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige Sticke,0
7,Okto Festival Ale,258,Altbier,8,Widmer Brothers Brewing Company,Notes:Classified as a German Altbier by the Widmer Brothers; brewed with an (ale) Alt yeast strain.availability: early Aug. - late Oct.bitterness : 28 IBUoriginal gravity: 13.5° PLATO,5.300000,3.460000,25,50,28,40,3,40,58,29,0,36,54,8,97,1,Widmer Brothers Brewing Company Okto Festival Ale,0
8,Southampton Altbier,259,Altbier,8,Southampton Publick House,Notes:,5.000000,3.600000,25,50,18,49,5,37,73,22,0,21,37,4,98,1,Southampton Publick House Southampton Altbier,1
9,Copper,260,Altbier,8,The Olde Mecklenburg Brewery,"Notes:OMB’s flagship, Copper, is an authentic Düsseldorf style “Altbier” or Amber Ale. There are no crazy flavors or gimmicks with Copper – its full-flavored start transitions to a smooth, balanced, easy-drinking finish. It’s the beer that tastes like beer.",4.800000,4.100000,25,50,25,35,4,38,39,13,1,8,60,16,97,1,The Olde Mecklenburg Brewery Copper,0
10,Organic Münster Alt,261,Altbier,8,Br

In [44]:
# Create new data frame (`beer_no_scores`) listing brews on `df_profile` with no exact match on `df_scores`
beer_no_scores = pd.DataFrame(df_profile[df_profile['beer_review_exists']==0]['Beer Name (Full)']).set_axis(['Beer Name (Full)'], axis=1)
beer_no_scores

,Beer Name (Full)
0,Alaskan Brewing Co. Amber
3,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...
6,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...
7,Widmer Brothers Brewing Company Okto Festival Ale
9,The Olde Mecklenburg Brewery Copper
...,...
5551,Fremont Brewing Company Winter Ale
5552,RJ Rockers Brewing Company The First Snow Ale
5554,Anchor Brewing Company Our Special Ale 2019 (A...
5556,"Shmaltz Brewing Company He'Brew Hanukkah, Chan..."


In [45]:
# Create new column on `df_scores` indicating whether the complete brew name exists on `df_profile`
df_scores['beer_profile_exists'] = np.where(df_scores['beer_name_full'].isin(list(df_profile['Beer Name (Full)'])), 1, 0)

# Create new data frame (`beer_no_profile`) listing brews on `df_scores` with no exact match on `df_profile`
beer_no_profile = pd.DataFrame(df_scores[df_scores['beer_profile_exists']==0]['beer_name_full']).set_axis(['beer_name_full'], axis=1)

In [ ]:
# # Get fuzzy matches for 2460 brews
# # Warning: Takes a long time to run
# fuzzy_match_beer_name = fuzzy_merge(beer_no_scores, beer_no_profile, 'Beer Name (Full)', 'beer_name_full', threshold=87, limit=1)
# fuzzy_match_beer_name.to_csv(your_data_dir / 'Beer Name Fuzzy Match List.csv', index=False)

This time, we found 1088 correct beer name matches and saved them as a .csv file called "Beer Name Fuzzy Match List.csv". Let's replace the beer names in `df_scores` with their equivalents from `df_profile`.

In [46]:
fuzzy_match_beer_name = pd.read_csv(data_dir / 'beer-profile-and-ratings-data-set/Beer Name Fuzzy Match List.csv')
fuzzy_match_beer_name

,Beer Name (Full),matches
0,(512) Brewing Company (512) Pecan Porter(512) ...,(512) Brewing Company (512) Pecan Porter
1,(512) Brewing Company (512) Whiskey Barrel Age...,(512) Brewing Company (512) Whiskey Barrel Age...
2,Žatecký Pivovar Žatec Dark Lager,Žatecký Pivovar Žatec Dark Lager
3,21st Amendment Brewery Brew Free Or Die IPA,21st Amendment Brewery 21st Amendment IPA
4,21st Amendment Brewery Lower De Boom Barleywine,21st Amendment Brewery Lower Da Boom Barleywine
...,...,...
1083,Wychwood Brewery Company Ltd Bah Humbug! Chris...,Wychwood Brewery Company Ltd Bah Humbug
1084,Wychwood Brewery Company Ltd Hobgoblin Ruby,Wychwood Brewery Company Ltd Hobgoblin
1085,Yards Brewing Co. Washington's Porter,Yards Brewing Co. General Washington's Tavern ...
1086,Zywiec Breweries PLC (Heineken) Tatra MocneZyw...,Zywiec Breweries PLC (Heineken) Tatra


In [47]:
# Save table as dictionary
fuzzy_match_dict = dict(zip(fuzzy_match_beer_name['Beer Name (Full)'], fuzzy_match_beer_name['matches']))

# Replace relevant brewery names in `df_profile`
df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace(fuzzy_match_dict)

In [48]:
# Additionally, manually replace some beer names (containing spacing errors) in both `df_profile` and `df_scores`
df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Panil  Barriquée (Sour Version)': 'Panil Barriquée (Sour Version)'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Panil Panil Barriquée (Sour Version)': 'Panil Barriquée (Sour Version)'})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Panil  Barriquée (Italy Version)': 'Panil Barriquée (Italy Version)'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Panil Panil Barriquée  (Italy Version)': 'Panil Barriquée (Italy Version)'})
                                                                                                                                   
df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Unibroue  17 Grande Réserve': 'Unibroue 17 Grande Réserve'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Unibroue Unibroue 17 Grande Réserve': 'Unibroue 17 Grande Réserve'})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Schorschbräu  Schorschbock 57%': 'Schorschbräu Schorschbock 57%'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Schorschbräu Schorschbräu Schorschbock 57%': 'Schorschbräu Schorschbock 57%'})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({"Alexander Keith's  India Pale Ale": "Alexander Keith's India Pale Ale"})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({"Alexander Keith's Alexander Keith's India Pale Ale": "Alexander Keith's India Pale Ale"})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Schorschbräu  Schorschbock 31%': 'Schorschbräu Schorschbock 31%'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Schorschbräu Schorschbräu Schorschbock 31%': 'Schorschbräu Schorschbock 31%'})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Klosterbrauerei Andechs Andechser Dunkles Weissbier': 'Klosterbrauerei Andechs Andechs Weissbier Dunkel'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Klosterbrauerei Andechs Andechser  Dunkles Weissbier': 'Klosterbrauerei Andechs Andechs Weissbier Dunkel'})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({"St. Georgenbräu St. Georgenbräu Keller Bier": "St. Georgenbräu Buttenheimer Keller Bier"})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({"St. Georgenbräu St. Georgenbräu  Keller Bier": "St. Georgenbräu Buttenheimer Keller Bier"})

df_profile['Beer Name (Full)'] = df_profile['Beer Name (Full)'].replace({'Weisses Bräuhaus G. Schneider & Sohn GmbH Schneider Weisse Mein Alkoholfreies': 'Weisses Bräuhaus G. Schneider & Sohn GmbH Schneider Weisse Mein Alkoholfreies'})
df_scores['beer_name_full'] = df_scores['beer_name_full'].replace({'Weisses Bräuhaus G. Schneider & Sohn GmbH Schneider Weisse  Mein Alkoholfreies': 'Weisses Bräuhaus G. Schneider & Sohn GmbH Schneider Weisse Mein Alkoholfreies'})

In [49]:
# Correcting 'beer_review_exist' indicator column values on `df_profile` after using fuzzy matches
df_profile['beer_review_exists'] = np.where(df_profile['Beer Name (Full)'].isin(list(df_scores['beer_name_full'].unique())), 1, 0)

# Correcting 'beer_profile_exists' indicator column values on `df_scores` after using fuzzy matches
df_scores['beer_profile_exists'] = np.where(df_scores['beer_name_full'].isin(list(df_profile['Beer Name (Full)'].unique())), 1, 0)

At this stage, we can remove all brews with incomplete data (i.e., those with no entry matches between `df_profile` and `df_scores`).

In [50]:
# Remove all brews that are not listed in either `df_profile` or `df_scores`
df_scores.drop(df_scores[df_scores['beer_profile_exists']==0].index, axis=0, inplace=True)
df_profile.drop(df_profile[df_profile['beer_review_exists']==0].index, axis=0, inplace=True)

In [51]:
len(df_scores.index)

3197

In [52]:
len(df_profile.index)

3197

Fantastic! We have an identical number of rows from each data frame. We also know the brew name labels are consistent on both data frames now. All we need to do now is to join all the columns we want to have in our new dataseet.

# Joining Datasets

In [53]:
# Drop columns that are redundant or no longer neccessary
df_profile.drop(['key', 'Style Key', 'brewery_review_exists', 'beer_review_exists', 'Ave Rating'], axis=1, inplace=True)
df_scores.drop(['brewery_name', 'beer_name', 'brewery_profile_exists', 'beer_profile_exists'], axis=1, inplace=True)


df_scores.rename(columns={"beer_name_full": "Beer Name (Full)"}, inplace=True)

In [54]:
# Join data frames to make new dataset
df_final = pd.merge(df_profile, df_scores, how='left', on=['Beer Name (Full)'])

# Re-arrange column order
df_final = df_final[['Name', 'Style', 'Brewery', 'Beer Name (Full)', 
                     'Description', 'ABV', 'Min IBU', 'Max IBU', 
                     'Astringency', 'Body', 'Alcohol', 
                     'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty', 
                     'review_aroma','review_appearance', 'review_palate', 'review_taste', 'number_of_reviews', 'review_overall']]

We ended up with a total of 3197 unique brews from 934 different breweries with in one table:

In [55]:
# Resulting dataset:
df_final

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,number_of_reviews,review_overall
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,497,3.847082
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,481,4.034304
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,377,3.830239
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,368,4.005435
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,96,3.817708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,Winter Shredder,Winter Warmer,Cisco Brewers Inc.,Cisco Brewers Inc. Winter Shredder,Notes:,8.8,35,50,15,37,...,23,45,67,74,4.125000,3.875000,3.875000,3.750000,4,4.000000
3193,The First Snow Ale,Winter Warmer,RJ Rockers Brewing Company,RJ Rockers Brewing Company First Snow Ale,Notes:This hearty American pale ale contains a...,6.0,35,50,15,31,...,54,14,140,58,3.950000,3.750000,3.760000,3.790000,50,3.730000
3194,Red Nose Winter Ale,Winter Warmer,Natty Greene's Pub & Brewing Co.,Natty Greene's Pub & Brewing Co. Red Nose Wint...,Notes:,6.8,35,50,8,44,...,26,21,96,77,3.576923,3.711538,3.596154,3.673077,26,3.711538
3195,Fish Tale Winterfish,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,Fish Brewing Company / Fishbowl Brewpub Fish T...,Notes:,7.5,35,50,11,36,...,81,110,18,73,3.902299,3.885057,3.862069,3.902299,87,3.879310


In [56]:
# df_final.to_csv(your_data_dir / 'beer_profile_and_ratings.csv', index=False)

The complete resulting dataset is available [here](https://www.kaggle.com/ruthgn/beer-profile-and-ratings-data-set).